<a href="https://colab.research.google.com/github/Guilherm0/Spark/blob/main/SparkCases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **SETUP**

## Spark UI

In [ ]:
!pip install -q pyspark
!pip install -q pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 761.3/761.3 KB 35.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.config('spark.ui.port', '4050').getOrCreate()
spark

In [ ]:
# Fazer login no site https://dashboard.ngrok.com/get-started/setup para obter autenticação própria
ngrok_token = 

In [ ]:
get_ipython().system_raw(f'ngrok authtoken {ngrok_token}')
get_ipython().system_raw('ngrok http 4050 &')
!sleep 3
print('URL para interface Spark:')
!curl -s http://localhost:4040/api/tunnels | grep -Po 'public_url":"(?=https)\K[^"]*'

URL para interface Spark:
https://7eda-34-86-89-111.ngrok.io


## Libraries

In [ ]:
import pandas as pd
from google.colab import files

import pyspark.sql.functions as F
from pyspark.sql.window import Window

# Load data

In [ ]:
# Load file
_ = files.upload()

Saving vgsales.csv to vgsales.csv


## Data Description

This dataset contains records of popular video games in North America, Japan, Europe and other parts of the world. Every video game in this dataset has at least 100k global sales.

[Source](https://www.kaggle.com/datasets/gregorut/videogamesales/code?datasetId=284&sortBy=voteCount) of dataset.

## Data Dictionary

| Column       | Explanation                                               |
|:-------------|:----------------------------------------------------------|
| Rank         | Ranking of overall sales                                  |
| Name         | Name of the game                                          |
| Platform     | Platform of the games release (i.e. PC,PS4, etc.)         |
| Year         | Year the game was released in                             |
| Genre        | Genre of the game                                         |
| Publisher    | Publisher of the game                                     |
| NA_Sales     | Number of sales in North America (in millions)            |
| EU_Sales     | Number of sales in Europe (in millions)                   |
| JP_Sales     | Number of sales in Japan (in millions)                    |
| Other_Sales  | Number of sales in other parts of the world (in millions) |
| Global_Sales | Number of total sales (in millions)                       |

## Data Preview

In [ ]:
# Mostrando os dados com pandas
pd.read_csv('vgsales.csv')

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37
...,...,...,...,...,...,...,...,...,...,...,...
16593,16596,Woody Woodpecker in Crazy Castle 5,GBA,2002.0,Platform,Kemco,0.01,0.00,0.00,0.00,0.01
16594,16597,Men in Black II: Alien Escape,GC,2003.0,Shooter,Infogrames,0.01,0.00,0.00,0.00,0.01
16595,16598,SCORE International Baja 1000: The Official Game,PS2,2008.0,Racing,Activision,0.00,0.00,0.00,0.00,0.01
16596,16599,Know How 2,DS,2010.0,Puzzle,7G//AMES,0.00,0.01,0.00,0.00,0.01


# Questões

## Questão 1

Leia os dados com Spark, certificando-se que

1. A tabela tenha duas partições e

2. As colunas da tabela tenham este respectivo schema:

|Column|Data type|
|---|---|
|Rank|integer|
|Name|string|
|Platform|string|
|Year|integer|
|Genre|string|
|Publisher|string|
|NA_Sales|double|
|EU_Sales|double|
|JP_Sales|double|
|Other_Sales|double|
|Global_Sales|double|

In [ ]:
df = spark.read.csv('vgsales.csv', header=True, inferSchema=True)
df = df.withColumn('Year', F.col('Year').cast('int'))
df = df.repartition(2)

In [ ]:
print('Num partitions:', df.rdd.getNumPartitions())
df.printSchema()

Num partitions: 2
root
 |-- Rank: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Platform: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- NA_Sales: double (nullable = true)
 |-- EU_Sales: double (nullable = true)
 |-- JP_Sales: double (nullable = true)
 |-- Other_Sales: double (nullable = true)
 |-- Global_Sales: double (nullable = true)



## Questão 2

Construa uma tabela com:
- contagem das observações
- média
- desvio padrão
- valor mínimo
- valor máximo

para todas as coluna de vendas (que possuem "Sales" no nome).

**Dica:** uma **única função** consegue calcular todos estes valores.

In [ ]:
# option 1
sales_columns = ['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales']
df.select(sales_columns).describe().show()

+-------+------------------+-------------------+-------------------+--------------------+------------------+
|summary|          NA_Sales|           EU_Sales|           JP_Sales|         Other_Sales|      Global_Sales|
+-------+------------------+-------------------+-------------------+--------------------+------------------+
|  count|             16598|              16598|              16598|               16598|             16598|
|   mean|0.2646674298108155|0.14665200626581024|0.07778166044101428|0.048063019640918934|0.5374406555006714|
| stddev|0.8166830292988791| 0.5053512312869121| 0.3092906480822032| 0.18858840291271461|1.5550279355699124|
|    min|               0.0|                0.0|                0.0|                 0.0|              0.01|
|    max|             41.49|              29.02|              10.22|               10.57|             82.74|
+-------+------------------+-------------------+-------------------+--------------------+------------------+



In [ ]:
# option 2
sales_columns = [col for col in df.columns if 'Sales' in col]
df.select(sales_columns).describe().show()

+-------+------------------+-------------------+-------------------+--------------------+------------------+
|summary|          NA_Sales|           EU_Sales|           JP_Sales|         Other_Sales|      Global_Sales|
+-------+------------------+-------------------+-------------------+--------------------+------------------+
|  count|             16598|              16598|              16598|               16598|             16598|
|   mean|0.2646674298108155|0.14665200626581024|0.07778166044101428|0.048063019640918934|0.5374406555006714|
| stddev|0.8166830292988791| 0.5053512312869121| 0.3092906480822032| 0.18858840291271461|1.5550279355699124|
|    min|               0.0|                0.0|                0.0|                 0.0|              0.01|
|    max|             41.49|              29.02|              10.22|               10.57|             82.74|
+-------+------------------+-------------------+-------------------+--------------------+------------------+



## Questão 3

Para cada variável categórica do tipo string, calcule quantas categorias distintas estão presentes.

In [ ]:
# Option 1
df.select(
    F.count_distinct('Name'),
    F.count_distinct('Platform'),
    F.count_distinct('Genre'),
    F.count_distinct('Publisher')
).show()

+--------------------+------------------------+---------------------+-------------------------+
|count(DISTINCT Name)|count(DISTINCT Platform)|count(DISTINCT Genre)|count(DISTINCT Publisher)|
+--------------------+------------------------+---------------------+-------------------------+
|               11493|                      31|                   12|                      579|
+--------------------+------------------------+---------------------+-------------------------+



In [ ]:
# Option 2
string_cols = ['Name','Platform','Genre','Publisher']
df.select([F.count_distinct(col) for col in string_cols]).show()

+--------------------+------------------------+---------------------+-------------------------+
|count(DISTINCT Name)|count(DISTINCT Platform)|count(DISTINCT Genre)|count(DISTINCT Publisher)|
+--------------------+------------------------+---------------------+-------------------------+
|               11493|                      31|                   12|                      579|
+--------------------+------------------------+---------------------+-------------------------+



In [ ]:
# Option 3
string_cols = [field['name'] for field in df.schema.jsonValue()['fields'] if field['type']=='string']
df.select([F.count_distinct(col) for col in string_cols]).show()

+--------------------+------------------------+---------------------+-------------------------+
|count(DISTINCT Name)|count(DISTINCT Platform)|count(DISTINCT Genre)|count(DISTINCT Publisher)|
+--------------------+------------------------+---------------------+-------------------------+
|               11493|                      31|                   12|                      579|
+--------------------+------------------------+---------------------+-------------------------+



**Resposta:** A região da Europa e "Outras regiões" tiverem um aumento notável quando comparado ao ínicio da década de 90.

## Questão 4

Qual plataforma vendeu mais, mundialmente, considerando todo o período histórico diponível dos dados?  
Mostre as top 10 plataformas em vendas.

In [ ]:
(
  df
 .groupby('Platform')
 .agg(F.sum('Global_Sales').alias('global_sales_sum'))
 .orderBy(F.desc('global_sales_sum'))
 .limit(10)
 .show()
 )

+--------+------------------+
|Platform|  global_sales_sum|
+--------+------------------+
|     PS2|1255.6399999999956|
|    X360| 979.9600000000003|
|     PS3| 957.8400000000025|
|     Wii| 926.7099999999973|
|      DS| 822.4899999999982|
|      PS| 730.6599999999992|
|     GBA|318.49999999999943|
|     PSP| 296.2799999999996|
|     PS4| 278.0999999999999|
|      PC|258.82000000000085|
+--------+------------------+



## Questão 5

Faça uma tabela com os jogos que aparecem em múltiplas plataformas, ordene de forma que os jogos com mais plataformas apareçam primeiro e responda:
- Qual jogo aparece em mais plataformas? Em quantas plataformas?

In [ ]:
(
 df
 .groupby('Name')
 .agg(F.countDistinct('Platform').alias('n_platform'))
 .orderBy(F.desc('n_platform'))
 .limit(10)
 .show(truncate=False)
)

+----------------------------+----------+
|Name                        |n_platform|
+----------------------------+----------+
|Need for Speed: Most Wanted |10        |
|LEGO Marvel Super Heroes    |9         |
|Madden NFL 07               |9         |
|Ratatouille                 |9         |
|FIFA 14                     |9         |
|Lego Batman 3: Beyond Gotham|8         |
|Terraria                    |8         |
|Monopoly                    |8         |
|LEGO Jurassic World         |8         |
|Madden NFL 08               |8         |
+----------------------------+----------+



**Resposta:** Need for Speed: Most Wanted. Aparece em 10 plataformas.

## Questão 6

Utilize a API do Pandas no Spark para calcular a soma das vendas globais para cada ano e gênro de jogo. Faça então um gráfico de linhas com os anos no eixo `x`, as vendas no eixo `y`, de forma que cada linha corresponda a um gênero de jogo.

**Dica:** após o cálculo, passar os dados para Pandas antes da plotagem, ou plotar diretamente aproveitando os métodos da classe `pyspark.pandas.DataFrame`.

Analise o gráfico e responda:
- Entre 1980 e 1990, quais gêneros mais venderam?
- Entre 2000 e 2015, quais gêneros mais venderam?

In [ ]:
(
  df
 .pandas_api()
 .groupby(['Year','Genre'])[['Global_Sales']].sum()
 .sort_index()
 .reset_index()
 .plot.line(x='Year', y='Global_Sales', color='Genre')
 .update_layout(title='Genre global sales through time', template='plotly_white')
)

**Resposta:**
- Entre 1980 e 1990 sobressaíram as vendas de plataforma, tiro (shooter) e puzzle.
- Entre 2000 e 2015 teve vendas elevadas de jogos de ação e esporte.

## Questão 7

Registre a tabela usando `createOrReplaceTempView` e faça uso da linguagem SQL criar uma tabela que:
- Considere apenas os anos da década de 90 e 
- Agrupe por ano para responder quantos % cada região teve do total de vendas (vendas globais).

Salve o resultado desta query em uma variável chamada `df_questao7`.

Após isso, execute o seguinte comando `df_questao7.pandas_api().set_index('YEAR').style.background_gradient(cmap='Oranges')` e responda:
- Qual/quais regiões tiveram, relativamente (comparado às vendas globais), mais vendas no fim da década de noventa do que no início?








In [ ]:
df.createOrReplaceTempView('table')

query = """
SELECT
  YEAR,
  ROUND(SUM(NA_Sales) / SUM(Global_Sales) * 100, 2) AS NA_Sales_PCT,
  ROUND(SUM(EU_Sales) / SUM(Global_Sales) * 100, 2) AS EU_Sales_PCT,
  ROUND(SUM(JP_Sales) / SUM(Global_Sales) * 100, 2) AS JP_Sales_PCT,
  ROUND(SUM(Other_Sales) / SUM(Global_Sales) * 100, 2) AS Other_Sales_PCT
FROM
  table
WHERE
  YEAR BETWEEN 1990 AND 1999
GROUP BY
  YEAR
ORDER BY
  YEAR
"""

df_questao7 = spark.sql(query)
df_questao7.show()

+----+------------+------------+------------+---------------+
|YEAR|NA_Sales_PCT|EU_Sales_PCT|JP_Sales_PCT|Other_Sales_PCT|
+----+------------+------------+------------+---------------+
|1990|       51.55|       15.45|       30.13|           2.83|
|1991|       39.59|       12.26|       45.86|            2.3|
|1992|       44.47|       15.38|       37.96|           2.17|
|1993|       32.88|       10.11|       55.09|           1.94|
|1994|       35.56|       18.79|       42.93|           2.78|
|1995|       28.17|       16.91|       51.92|            3.0|
|1996|       43.57|       23.73|       28.84|           3.86|
|1997|       47.14|       24.04|       24.32|           4.54|
|1998|       50.05|       26.08|       19.51|            4.3|
|1999|       50.17|       24.94|       20.83|            4.0|
+----+------------+------------+------------+---------------+



In [ ]:
df_questao7.pandas_api().set_index('YEAR').style.background_gradient(cmap='Oranges')

,NA_Sales_PCT,EU_Sales_PCT,JP_Sales_PCT,Other_Sales_PCT
YEAR,,,,
1990,51.550000,15.450000,30.130000,2.830000
1991,39.590000,12.260000,45.860000,2.300000
1992,44.470000,15.380000,37.960000,2.170000
1993,32.880000,10.110000,55.090000,1.940000
1994,35.560000,18.790000,42.930000,2.780000
1995,28.170000,16.910000,51.920000,3.000000
1996,43.570000,23.730000,28.840000,3.860000
1997,47.140000,24.040000,24.320000,4.540000
1998,50.050000,26.080000,19.510000,4.300000


## Questão 8

Calcule quantas vezes cada região teve vendas superiores às demais.

In [ ]:
regions = ['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales']

(
  df
 .withColumn('max_sale', F.greatest(*regions))
 .withColumn(
     'greatest_sale_region',
     F.when(F.col('NA_Sales')==F.col('max_sale'), 'NA_Sales')
     .when(F.col('EU_Sales')==F.col('max_sale'), 'EU_Sales')
     .when(F.col('JP_Sales')==F.col('max_sale'), 'JP_Sales')
     .otherwise('Other_Sales')
 )
 .groupby('greatest_sale_region')
 .count()
 .show()
)

+--------------------+-----+
|greatest_sale_region|count|
+--------------------+-----+
|            EU_Sales| 2380|
|            JP_Sales| 4029|
|            NA_Sales|10113|
|         Other_Sales|   76|
+--------------------+-----+



## Questão 9

Construe uma tabela que mostre a diferença do total de vendas em um década com a década anterior e responda:
- Qual década apresentou a **menor diferença** comparada à década anterior?

In [ ]:
w = Window().orderBy('Decade')

(
  df
 .where('Year IS NOT NULL')
 .withColumn('Decade', F.floor(F.col('Year')/10)*10) # possível fazer com when, porém mais código necessário
 .groupby('Decade')
 .agg(F.sum('Global_Sales').alias('global_sales_sum'))
 .withColumn('global_sales_sum_lag', F.lag('global_sales_sum').over(w))
 .withColumn('Delta_global_sales_sum', F.col('global_sales_sum') - F.col('global_sales_sum_lag'))
 .orderBy(F.abs('Decade'))
 .show()
)

+------+------------------+--------------------+----------------------+
|Decade|  global_sales_sum|global_sales_sum_lag|Delta_global_sales_sum|
+------+------------------+--------------------+----------------------+
|  1980|376.58000000000004|                null|                  null|
|  1990|1278.9099999999987|  376.58000000000004|     902.3299999999987|
|  2000| 4644.020000000007|  1278.9099999999987|     3365.110000000008|
|  2010| 2520.559999999984|   4644.020000000007|    -2123.460000000023|
|  2020|              0.29|   2520.559999999984|    -2520.269999999984|
+------+------------------+--------------------+----------------------+



**Resposta:**
- A década de 90 teve a menor diferença de vendas com a década anterior: ~902M